In [1]:
import gymnasium as gym
import numpy as np

In [2]:
env = gym.make("LunarLander-v3")
# observation, info = env.reset()
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample())
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample())

C:\Users\ayush\AppData\Roaming\Python\Python313\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


_____OBSERVATION SPACE_____ 

Observation Space Shape (8,)
Sample observation [-0.28151593 -1.6177502  -4.772857    4.4483733  -0.6844411   3.9497395
  0.6303274   0.3603528 ]

 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 1


In [3]:
for episode in range(4):
    observation, info = env.reset()
    done = False
    score=0
    while not done:
        env.render()  # Render the environment
        action = env.action_space.sample()
        n_observation, reward, done,_,_ = env.step(action)
        score+= reward

    print(f"Episode: {episode + 1}, Action: {action}, Score: {score}")
env.close()  # Close the environment when done

Episode: 1, Action: 0, Score: -28.399937988288087
Episode: 2, Action: 0, Score: -128.20689433282766
Episode: 3, Action: 1, Score: -135.7640357825592
Episode: 4, Action: 2, Score: -122.72749129438401


C:\Users\ayush\AppData\Roaming\Python\Python313\site-packages\gymnasium\envs\box2d\lunar_lander.py:672: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("LunarLander-v3", render_mode="rgb_array")
  gym.logger.warn(


In [4]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.monitor import Monitor
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch

In [ ]:
# %% Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# %% Create the training environment
env = Monitor(gym.make("LunarLander-v3"))

# %% Initialize the model (on GPU if available)
model = PPO(
    policy="MlpPolicy",
    env=env,
    n_steps=1024,
    batch_size=64,
    n_epochs=4,
    gamma=0.999,
    gae_lambda=0.98,
    ent_coef=0.01,
    verbose=1,
    device=device  # 👈 Move model to GPU
)

# %% Track rewards manually
timesteps_per_iteration = 10000
total_timesteps = 1000000
reward_means = []
timestep_x = []

# %% Training with tqdm
for i in tqdm(range(total_timesteps // timesteps_per_iteration), desc="Training Progress"):
    model.learn(total_timesteps=timesteps_per_iteration, reset_num_timesteps=False)
    
    # Evaluate after each chunk
    eval_env = Monitor(gym.make("LunarLander-v3"))
    mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=5, deterministic=True)
    
    reward_means.append(mean_reward)
    timestep_x.append((i + 1) * timesteps_per_iteration)

    print(f"[{(i+1)*timesteps_per_iteration}] Mean Reward: {mean_reward:.2f} ± {std_reward:.2f}")

# %% Save the trained model
model_name = "ppo-LunarLander-v3"
model.save(model_name)

# %% Plot the rewards
plt.plot(timestep_x, reward_means)
plt.xlabel("Timesteps")
plt.ylabel("Mean Reward")
plt.title("PPO LunarLander-v3 Training Reward Curve")
plt.grid()
plt.show()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [5]:
env= gym.make("LunarLander-v3")
# env = make_vec_env("LunarLander-v2", n_envs=16)
model = PPO.load("ppo-LunarLander-v3.zip")


In [ ]:
from stable_baselines3.common.monitor import Monitor
eval_env = Monitor(gym.make("LunarLander-v3",render_mode="human"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=249.26 +/- 47.84990896782074


: 